In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
train_joke_df = pd.read_csv(r'..\data\recsys-in-practice\train_joke_df.csv')

joke_df = pd.read_csv(r'..\surprise\joke_df_features_svd_best_4.02408.csv')
user_df = pd.read_csv(r'..\surprise\user_df_features_svd_best_4.02408.csv')

In [3]:
train_joke_df = train_joke_df.merge(user_df, left_on='UID', right_index=True).merge(joke_df, left_on='JID', right_index=True)
train_joke_df

,UID,JID,Rating,user_feature_1,user_feature_2,user_feature_3,user_feature_4,user_feature_5,user_feature_6,user_feature_7,...,joke_feature_503,joke_feature_504,joke_feature_505,joke_feature_506,joke_feature_507,joke_feature_508,joke_feature_509,joke_feature_510,joke_feature_511,joke_feature_512
0,18029,6,-1.26,-0.266763,0.007233,0.025731,-0.033462,0.081937,0.219742,-0.401678,...,0.247370,0.043730,0.476022,0.186204,-0.367366,-0.050056,0.364000,-0.110439,0.094803,-0.225325
120304,3366,6,3.35,-0.258486,0.165140,-0.101690,0.063709,-0.019856,-0.075516,0.093346,...,0.247370,0.043730,0.476022,0.186204,-0.367366,-0.050056,0.364000,-0.110439,0.094803,-0.225325
492994,12735,6,1.26,0.231300,-0.054370,0.240180,0.030590,0.243780,-0.370554,-0.146181,...,0.247370,0.043730,0.476022,0.186204,-0.367366,-0.050056,0.364000,-0.110439,0.094803,-0.225325
81927,11365,6,-5.87,0.109854,-0.032486,0.074281,0.045938,0.108607,-0.076549,-0.030610,...,0.247370,0.043730,0.476022,0.186204,-0.367366,-0.050056,0.364000,-0.110439,0.094803,-0.225325
650652,17990,6,-0.29,-0.227620,0.211211,0.256456,-0.138557,0.109195,0.012632,-0.298448,...,0.247370,0.043730,0.476022,0.186204,-0.367366,-0.050056,0.364000,-0.110439,0.094803,-0.225325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501169,19068,85,1.60,0.054417,-0.135585,0.021027,-0.111934,0.192451,-0.028916,-0.010914,...,0.019194,0.066531,0.105555,0.491504,0.279681,-0.212129,0.105872,-0.511717,0.070793,0.288640
326575,24910,85,-1.80,0.071639,0.305202,0.209195,-0.100342,-0.408026,-0.543737,0.179347,...,0.019194,0.066531,0.105555,0.491504,0.279681,-0.212129,0.105872,-0.511717,0.070793,0.288640
1040976,9017,85,3.93,0.236490,0.168426,0.284546,0.076836,-0.244758,-0.075950,0.071511,...,0.019194,0.066531,0.105555,0.491504,0.279681,-0.212129,0.105872,-0.511717,0.070793,0.288640
1148245,21968,85,3.54,-0.016625,-0.387227,0.206067,0.417567,-0.329962,0.242497,-0.083002,...,0.019194,0.066531,0.105555,0.491504,0.279681,-0.212129,0.105872,-0.511717,0.070793,0.288640


In [4]:
train_df, valid_df = train_test_split(train_joke_df, test_size=0.001, random_state=42)

In [5]:
# сделаем сортировку и перепишем index
train_df = train_df.sort_values(by=['UID', 'JID'])
train_df = train_df.reset_index(drop=True)

valid_df = valid_df.sort_values(by=['UID', 'JID'])
valid_df = valid_df.reset_index(drop=True)

In [6]:
from catboost import CatBoostRanker, Pool, MetricVisualizer, CatBoostRegressor
from copy import deepcopy

In [7]:
cat_features = ['UID', 'JID']

In [8]:
train_pool = Pool(train_df.drop(columns='Rating'), label=train_df['Rating'], group_id=train_df['UID'],cat_features=cat_features)
valid_pool = Pool(valid_df.drop(columns='Rating'), label=valid_df['Rating'], group_id=valid_df['UID'],cat_features=cat_features)

In [15]:
default_parameters = {
    'iterations': 3000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE'
}


In [16]:
model = CatBoostRegressor(**default_parameters)
model.fit(train_pool, eval_set=valid_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.081595
0:	learn: 5.1400033	test: 5.0651139	best: 5.0651139 (0)	total: 1.26s	remaining: 1h 3m 12s
1:	learn: 5.0592518	test: 4.9827526	best: 4.9827526 (1)	total: 3.15s	remaining: 1h 18m 43s
2:	learn: 4.9892677	test: 4.9118421	best: 4.9118421 (2)	total: 4.11s	remaining: 1h 8m 30s
3:	learn: 4.9296078	test: 4.8504321	best: 4.8504321 (3)	total: 5.07s	remaining: 1h 3m 18s
4:	learn: 4.8770848	test: 4.7964130	best: 4.7964130 (4)	total: 6.02s	remaining: 1h 5s
5:	learn: 4.8303046	test: 4.7475929	best: 4.7475929 (5)	total: 7.89s	remaining: 1h 5m 35s
6:	learn: 4.7892546	test: 4.7057119	best: 4.7057119 (6)	total: 8.84s	remaining: 1h 2m 59s
7:	learn: 4.7544858	test: 4.6702953	best: 4.6702953 (7)	total: 10.4s	remaining: 1h 5m 5s
8:	learn: 4.7238657	test: 4.6410389	best: 4.6410389 (8)	total: 12s	remaining: 1h 6m 25s
9:	learn: 4.6973347	test: 4.6166512	best: 4.6166512 (9)	total: 12.9s	remaining: 1h 4m 4s
10:	learn: 4.6747597	test: 4.5931215	best: 4.5931215 (10)	total: 14.4s	remain

89:	learn: 4.5005149	test: 4.4167457	best: 4.4167457 (89)	total: 2m 4s	remaining: 1h 7m 13s
90:	learn: 4.5004163	test: 4.4167407	best: 4.4167407 (90)	total: 2m 6s	remaining: 1h 7m 15s
91:	learn: 4.5001906	test: 4.4165000	best: 4.4165000 (91)	total: 2m 8s	remaining: 1h 7m 32s
92:	learn: 4.5000174	test: 4.4163509	best: 4.4163509 (92)	total: 2m 9s	remaining: 1h 7m 15s
93:	learn: 4.4999093	test: 4.4163237	best: 4.4163237 (93)	total: 2m 10s	remaining: 1h 7m 5s
94:	learn: 4.4998304	test: 4.4161310	best: 4.4161310 (94)	total: 2m 11s	remaining: 1h 6m 59s
95:	learn: 4.4995674	test: 4.4158243	best: 4.4158243 (95)	total: 2m 12s	remaining: 1h 7m
96:	learn: 4.4994224	test: 4.4154242	best: 4.4154242 (96)	total: 2m 13s	remaining: 1h 6m 40s
97:	learn: 4.4992288	test: 4.4150479	best: 4.4150479 (97)	total: 2m 14s	remaining: 1h 6m 28s
98:	learn: 4.4991169	test: 4.4152709	best: 4.4150479 (97)	total: 2m 15s	remaining: 1h 6m 13s
99:	learn: 4.4989755	test: 4.4155574	best: 4.4150479 (97)	total: 2m 16s	remaini

176:	learn: 4.4899146	test: 4.4046580	best: 4.4046580 (176)	total: 3m 51s	remaining: 1h 1m 27s
177:	learn: 4.4898203	test: 4.4043100	best: 4.4043100 (177)	total: 3m 52s	remaining: 1h 1m 20s
178:	learn: 4.4896374	test: 4.4042486	best: 4.4042486 (178)	total: 3m 53s	remaining: 1h 1m 17s
179:	learn: 4.4895523	test: 4.4041959	best: 4.4041959 (179)	total: 3m 54s	remaining: 1h 1m 9s
180:	learn: 4.4894587	test: 4.4042328	best: 4.4041959 (179)	total: 3m 54s	remaining: 1h 59s
181:	learn: 4.4893183	test: 4.4040847	best: 4.4040847 (181)	total: 3m 56s	remaining: 1h 59s
182:	learn: 4.4892135	test: 4.4040111	best: 4.4040111 (182)	total: 3m 57s	remaining: 1h 54s
183:	learn: 4.4890508	test: 4.4040100	best: 4.4040100 (183)	total: 3m 58s	remaining: 1h 55s
184:	learn: 4.4889417	test: 4.4037817	best: 4.4037817 (184)	total: 3m 59s	remaining: 1h 49s
185:	learn: 4.4888161	test: 4.4037341	best: 4.4037341 (185)	total: 4m	remaining: 1h 44s
186:	learn: 4.4886710	test: 4.4034552	best: 4.4034552 (186)	total: 4m 1s	

265:	learn: 4.4780783	test: 4.3927006	best: 4.3927006 (265)	total: 5m 38s	remaining: 58m 2s
266:	learn: 4.4778406	test: 4.3928125	best: 4.3927006 (265)	total: 5m 41s	remaining: 58m 10s
267:	learn: 4.4777622	test: 4.3930557	best: 4.3927006 (265)	total: 5m 42s	remaining: 58m 12s
268:	learn: 4.4776803	test: 4.3934084	best: 4.3927006 (265)	total: 5m 43s	remaining: 58m 9s
269:	learn: 4.4775276	test: 4.3933365	best: 4.3927006 (265)	total: 5m 45s	remaining: 58m 9s
270:	learn: 4.4774029	test: 4.3929511	best: 4.3927006 (265)	total: 5m 46s	remaining: 58m 9s
271:	learn: 4.4772690	test: 4.3928652	best: 4.3927006 (265)	total: 5m 47s	remaining: 58m 7s
272:	learn: 4.4771823	test: 4.3928016	best: 4.3927006 (265)	total: 5m 48s	remaining: 58m 2s
273:	learn: 4.4770592	test: 4.3925672	best: 4.3925672 (273)	total: 5m 49s	remaining: 58m
274:	learn: 4.4769135	test: 4.3924836	best: 4.3924836 (274)	total: 5m 51s	remaining: 57m 59s
275:	learn: 4.4767946	test: 4.3925372	best: 4.3924836 (274)	total: 5m 52s	remain

...

2608:	learn: 4.3240266	test: 4.3053001	best: 4.3035363 (2544)	total: 56m 35s	remaining: 8m 28s
2609:	learn: 4.3239843	test: 4.3053173	best: 4.3035363 (2544)	total: 56m 36s	remaining: 8m 27s
2610:	learn: 4.3239430	test: 4.3051564	best: 4.3035363 (2544)	total: 56m 37s	remaining: 8m 26s
2611:	learn: 4.3239018	test: 4.3052260	best: 4.3035363 (2544)	total: 56m 38s	remaining: 8m 24s
2612:	learn: 4.3238609	test: 4.3052627	best: 4.3035363 (2544)	total: 56m 39s	remaining: 8m 23s
2613:	learn: 4.3238204	test: 4.3053342	best: 4.3035363 (2544)	total: 56m 40s	remaining: 8m 22s
2614:	learn: 4.3237769	test: 4.3053412	best: 4.3035363 (2544)	total: 56m 41s	remaining: 8m 20s
2615:	learn: 4.3237380	test: 4.3053516	best: 4.3035363 (2544)	total: 56m 43s	remaining: 8m 19s
2616:	learn: 4.3236925	test: 4.3053420	best: 4.3035363 (2544)	total: 56m 44s	remaining: 8m 18s
2617:	learn: 4.3236505	test: 4.3054384	best: 4.3035363 (2544)	total: 56m 46s	remaining: 8m 17s
2618:	learn: 4.3236009	test: 4.3053841	best: 4.303

2695:	learn: 4.3199545	test: 4.3047065	best: 4.3035363 (2544)	total: 58m 26s	remaining: 6m 35s
2696:	learn: 4.3199146	test: 4.3046981	best: 4.3035363 (2544)	total: 58m 27s	remaining: 6m 34s
2697:	learn: 4.3198708	test: 4.3046571	best: 4.3035363 (2544)	total: 58m 29s	remaining: 6m 32s
2698:	learn: 4.3198345	test: 4.3046677	best: 4.3035363 (2544)	total: 58m 30s	remaining: 6m 31s
2699:	learn: 4.3198150	test: 4.3047310	best: 4.3035363 (2544)	total: 58m 30s	remaining: 6m 30s
2700:	learn: 4.3197741	test: 4.3046571	best: 4.3035363 (2544)	total: 58m 32s	remaining: 6m 28s
2701:	learn: 4.3197258	test: 4.3045329	best: 4.3035363 (2544)	total: 58m 33s	remaining: 6m 27s
2702:	learn: 4.3196844	test: 4.3045829	best: 4.3035363 (2544)	total: 58m 35s	remaining: 6m 26s
2703:	learn: 4.3196474	test: 4.3046995	best: 4.3035363 (2544)	total: 58m 36s	remaining: 6m 24s
2704:	learn: 4.3196004	test: 4.3044245	best: 4.3035363 (2544)	total: 58m 38s	remaining: 6m 23s
2705:	learn: 4.3195613	test: 4.3043197	best: 4.303

2782:	learn: 4.3162188	test: 4.3010660	best: 4.3010660 (2782)	total: 1h 14s	remaining: 4m 41s
2783:	learn: 4.3161802	test: 4.3011276	best: 4.3010660 (2782)	total: 1h 16s	remaining: 4m 40s
2784:	learn: 4.3161327	test: 4.3010587	best: 4.3010587 (2784)	total: 1h 17s	remaining: 4m 39s
2785:	learn: 4.3160964	test: 4.3010847	best: 4.3010587 (2784)	total: 1h 18s	remaining: 4m 37s
2786:	learn: 4.3160702	test: 4.3011121	best: 4.3010587 (2784)	total: 1h 19s	remaining: 4m 36s
2787:	learn: 4.3160292	test: 4.3009324	best: 4.3009324 (2787)	total: 1h 20s	remaining: 4m 35s
2788:	learn: 4.3160028	test: 4.3008974	best: 4.3008974 (2788)	total: 1h 21s	remaining: 4m 34s
2789:	learn: 4.3159562	test: 4.3006279	best: 4.3006279 (2789)	total: 1h 22s	remaining: 4m 32s
2790:	learn: 4.3159181	test: 4.3006621	best: 4.3006279 (2789)	total: 1h 23s	remaining: 4m 31s
2791:	learn: 4.3158746	test: 4.3004945	best: 4.3004945 (2791)	total: 1h 25s	remaining: 4m 30s
2792:	learn: 4.3158344	test: 4.3004498	best: 4.3004498 (2792

2868:	learn: 4.3120344	test: 4.3000581	best: 4.2998045 (2864)	total: 1h 2m 6s	remaining: 2m 50s
2869:	learn: 4.3119950	test: 4.3001251	best: 4.2998045 (2864)	total: 1h 2m 8s	remaining: 2m 48s
2870:	learn: 4.3119623	test: 4.3001991	best: 4.2998045 (2864)	total: 1h 2m 8s	remaining: 2m 47s
2871:	learn: 4.3119215	test: 4.3002109	best: 4.2998045 (2864)	total: 1h 2m 10s	remaining: 2m 46s
2872:	learn: 4.3118740	test: 4.3002061	best: 4.2998045 (2864)	total: 1h 2m 11s	remaining: 2m 44s
2873:	learn: 4.3118366	test: 4.3001417	best: 4.2998045 (2864)	total: 1h 2m 12s	remaining: 2m 43s
2874:	learn: 4.3118015	test: 4.3000365	best: 4.2998045 (2864)	total: 1h 2m 14s	remaining: 2m 42s
2875:	learn: 4.3117596	test: 4.2999979	best: 4.2998045 (2864)	total: 1h 2m 15s	remaining: 2m 41s
2876:	learn: 4.3117176	test: 4.2998901	best: 4.2998045 (2864)	total: 1h 2m 17s	remaining: 2m 39s
2877:	learn: 4.3116775	test: 4.2998489	best: 4.2998045 (2864)	total: 1h 2m 18s	remaining: 2m 38s
2878:	learn: 4.3115885	test: 4.29

2953:	learn: 4.3083798	test: 4.2995324	best: 4.2994759 (2944)	total: 1h 3m 58s	remaining: 59.8s
2954:	learn: 4.3083259	test: 4.2994497	best: 4.2994497 (2954)	total: 1h 3m 59s	remaining: 58.5s
2955:	learn: 4.3082895	test: 4.2995230	best: 4.2994497 (2954)	total: 1h 4m	remaining: 57.2s
2956:	learn: 4.3082442	test: 4.2995587	best: 4.2994497 (2954)	total: 1h 4m 1s	remaining: 55.9s
2957:	learn: 4.3082072	test: 4.2994812	best: 4.2994497 (2954)	total: 1h 4m 3s	remaining: 54.6s
2958:	learn: 4.3081673	test: 4.2995332	best: 4.2994497 (2954)	total: 1h 4m 4s	remaining: 53.3s
2959:	learn: 4.3081234	test: 4.2996389	best: 4.2994497 (2954)	total: 1h 4m 6s	remaining: 52s
2960:	learn: 4.3081013	test: 4.2996290	best: 4.2994497 (2954)	total: 1h 4m 6s	remaining: 50.7s
2961:	learn: 4.3080636	test: 4.2997994	best: 4.2994497 (2954)	total: 1h 4m 7s	remaining: 49.4s
2962:	learn: 4.3080389	test: 4.2998727	best: 4.2994497 (2954)	total: 1h 4m 9s	remaining: 48.1s
2963:	learn: 4.3079937	test: 4.2997769	best: 4.299449

In [18]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))

predict = model.predict(train_pool)
print(mean_squared_error(train_df['Rating'].values, predict, squared=False))


4.298839106596443
4.128421035050222


In [19]:
model_name = "catboost_svd_best"

In [20]:
model.save_model(model_name)

from_file = CatBoostRegressor()

from_file.load_model(model_name)


In [22]:
predict = from_file.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))

predict = from_file.predict(train_pool)
print(mean_squared_error(train_df['Rating'].values, predict, squared=False))


4.298839106596443
4.128421035050222


In [ ]:
assert False

In [26]:
import gc

In [31]:
gc.collect()

0

In [30]:
del train_pool

In [42]:
test_joke_df_nofactrating = pd.read_csv(r'..\data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)

test_joke_df_nofactrating = test_joke_df_nofactrating.merge(user_df, how='left', left_on='UID', right_index=True).merge(joke_df, how='left', left_on='JID', right_index=True)
test_joke_df_nofactrating
test_joke_df_nofactrating = test_joke_df_nofactrating.sort_values(by=['UID', 'JID'])
test_pool = Pool(test_joke_df_nofactrating, group_id=test_joke_df_nofactrating['UID'], cat_features=cat_features)

In [44]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost_svd_best.csv')

,Rating
InteractionID,
219361,-1.741032
83962,-3.116634
64748,-3.917405
83931,-0.679487
170573,-2.813527


In [45]:
test_joke_df_nofactrating

,UID,JID,user_feature_1,user_feature_2,user_feature_3,user_feature_4,user_feature_5,user_feature_6,user_feature_7,user_feature_8,...,joke_feature_504,joke_feature_505,joke_feature_506,joke_feature_507,joke_feature_508,joke_feature_509,joke_feature_510,joke_feature_511,joke_feature_512,Rating
InteractionID,,,,,,,,,,,,,,,,,,,,,
219361,1,11,0.10013,0.011191,0.180694,0.057651,-0.506843,-0.004767,-0.203006,0.481219,...,0.343735,0.424234,0.160415,0.206512,0.019995,-0.028290,-0.238535,-0.588054,0.023758,-1.741032
83962,1,19,0.10013,0.011191,0.180694,0.057651,-0.506843,-0.004767,-0.203006,0.481219,...,-0.274956,0.423745,-0.096619,-0.290765,0.086046,0.177519,0.021229,-0.234629,-0.320996,-3.116634
64748,1,25,0.10013,0.011191,0.180694,0.057651,-0.506843,-0.004767,-0.203006,0.481219,...,-0.306707,0.080947,0.039174,0.029612,-0.411554,0.050813,-0.166340,-0.134812,0.034243,-3.917405
83931,1,31,0.10013,0.011191,0.180694,0.057651,-0.506843,-0.004767,-0.203006,0.481219,...,0.145702,0.228719,0.057454,0.207939,0.234427,0.330774,-0.314968,0.062298,-0.479028,-0.679487
170573,1,34,0.10013,0.011191,0.180694,0.057651,-0.506843,-0.004767,-0.203006,0.481219,...,-0.152723,-0.007713,0.008653,0.087951,0.116084,0.011031,-0.196337,0.437508,0.092751,-2.813527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291554,24983,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.301839,-0.011438,-0.280908,-0.054214,-0.036488,-0.014815,-0.162373,0.034628,0.369599,-4.611290
21758,24983,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.334802,0.067404,0.142401,-0.001793,-0.052562,0.345566,-0.109195,0.034905,0.226143,1.027993
14611,24983,63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.152483,-0.329221,0.223465,0.130147,0.205422,-0.050856,-0.208522,-0.312381,-0.278464,-3.587975
